In [1]:
import docplex
import pandas as pd
import tensorflow as tf
import numpy as np
import utility
import copy
import mlp_explainer
import mymetrics
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from milp import codify_network
from teste import get_minimal_explanation
from sklearn import metrics
import time
import dataframe_image as dfi

In [2]:
#Pima Dataset
df = pd.read_csv('./datasets/diabetes.csv')

scaler = MinMaxScaler()
scaler.fit(df.values[:, :-1])
scaled_df = scaler.transform(df.values[:, :-1])
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)
df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
df_scaled['target'] = targets
columns = df_scaled.columns
dir_path = 'Pima'
dataset_name = 'Pima'
display(df_scaled)

0.0 1.0
Original Targets:  [0. 1.] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,target
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.039710,0.700000,0
764,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.111870,0.100000,0
765,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.071307,0.150000,0
766,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.115713,0.433333,1


In [3]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.3,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

training_data = pd.DataFrame(X_train, columns = columns[:-1])
training_data[columns[-1]] = y_train
testing_data = pd.DataFrame(X_test, columns = columns[:-1])
testing_data[columns[-1]] = y_test
dataframe = pd.concat([training_data, testing_data])
data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

original_bounds = [[dataframe[dataframe.columns[i]].min(),dataframe[dataframe.columns[i]].max()] for i in range(len(dataframe.columns[:-1]))]
keras_model = tf.keras.models.load_model(f'new_models/{dataset_name}.h5')

In [4]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

In [5]:
predictions = []
possible_classes = np.unique(y_test)
class_indexes = []
class_predictions = []
for i in range(n_classes):
    class_indexes.append([])
    class_predictions.append([])
possible_classes, class_indexes, class_predictions
data = testing_data.to_numpy()
for i in range(len(data)):
    predictions.append(mlp_explainer.model_classification_output(k_model=keras_model, net_input=data[i, :-1])[1].numpy())    
    for j,p_class in enumerate(possible_classes):
        if predictions[-1] == p_class:
            class_indexes[j].append(i)
            class_predictions[j].append(data[i, :-1])
print("Accuracy Test Data:", metrics.accuracy_score(testing_data.to_numpy()[:, -1], predictions))

Accuracy Test Data: 0.7878787878787878


In [6]:
cols = list(testing_data.columns)
if 'target' not in cols:
    cols.append('target')
predicted_dataset = []
for i,pos_class in enumerate(np.unique(y_test)):
    for instance in (testing_data.to_numpy()[:, :-1][class_indexes[i]]):
        instance = np.append(instance, pos_class.astype('int'))
        predicted_dataset.append(instance)
predicted_dataset = np.asarray(predicted_dataset)
pred_dataset_df = pd.DataFrame(predicted_dataset, columns=cols)
pred_dataset_df['target'] = pred_dataset_df['target'].astype('int')

In [7]:
metrics_dataframes = []
times_onestep = []
sizes_onestep = []
rsum_onestep = []
coverage_onestep = []
pos_exp_onestep = []
neg_exp_onestep = []

In [8]:
original_bounds

[[0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999998],
 [0.0, 1.0]]

In [9]:
onestep_explanations = []
for j in range(len(pred_dataset_df['target'].unique())):
    for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):
        start = time.perf_counter()
        explanation, minimal = mlp_explainer.run_explanation(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, enable_log=False,
                                                             )
        end = time.perf_counter()
        onestep_explanations.append(explanation)
        times_onestep.append(end-start)
        sizes_onestep.append(len(minimal))
        rsum_onestep.append(mymetrics.range_sum(onestep_explanations[-1]))
        coverage_onestep.append(len(mymetrics.calculate_coverage(testing_data, onestep_explanations[-1])))

In [10]:
p_values = [0.50] #[0.25, 0.50, 0.75]
for p_value in p_values:
    print(f"p = {p_value}")
    times_twostep = []
    sizes_twostep = []
    rsum_twostep = []
    coverage_twostep = []
    twostep_explanations = []
    for j in range(len(pred_dataset_df['target'].unique())):
        for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):
            start = time.perf_counter()
            
            explanation, minimal = mlp_explainer.run_explanation_doublestep(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, p=p_value)
            end = time.perf_counter()
            twostep_explanations.append(explanation)
            times_twostep.append(end-start)
            sizes_twostep.append(len(minimal))
            rsum_twostep.append(mymetrics.range_sum(twostep_explanations[-1]))
            
            coverage_twostep.append(len(mymetrics.calculate_coverage(testing_data, twostep_explanations[-1])))

    time_mean_twostep = sum(times_twostep)/len(times_twostep)
    time_std_twostep = np.std(times_twostep)
    sizes_mean_twostep = sum(sizes_twostep)/len(sizes_twostep)
    sizes_std_twostep = np.std(sizes_twostep)
    rsum_mean_twostep = sum(rsum_twostep)/len(rsum_twostep)
    rsum_std_twostep = np.std(rsum_twostep)

    coverage_mean_twostep = sum(coverage_twostep)/len(coverage_twostep)
    coverage_std_twostep = np.std(coverage_twostep)

    time_mean_onestep = sum(times_onestep)/len(times_onestep)
    time_std_onestep = np.std(times_onestep)
    sizes_mean_onestep = sum(sizes_onestep)/len(sizes_onestep)
    sizes_std_onestep = np.std(sizes_onestep)
    rsum_mean_onestep = sum(rsum_onestep)/len(rsum_onestep)
    rsum_std_onestep = np.std(rsum_onestep)

    coverage_mean_onestep = sum(coverage_onestep)/len(coverage_onestep)
    coverage_std_onestep = np.std(coverage_onestep)

    all_metrics_names = ['Metric','Onestep_MEAN','Onestep_STD','Twostep_MEAN','Twostep_STD']

    all_metrics_mean_df  = pd.DataFrame(columns=all_metrics_names)
    pattern_row = ['Time',time_mean_onestep, time_std_onestep, time_mean_twostep,time_std_twostep]
    all_metrics_mean_df.loc[len(all_metrics_mean_df), :] = pattern_row

    pattern_row = ['Size', sizes_mean_onestep, sizes_std_onestep, sizes_mean_twostep,sizes_std_twostep]
    all_metrics_mean_df.loc[len(all_metrics_mean_df), :] = pattern_row

    pattern_row = ['Ranges_Sum', rsum_mean_onestep, rsum_std_onestep, rsum_mean_twostep,rsum_std_twostep]
    all_metrics_mean_df.loc[len(all_metrics_mean_df), :] = pattern_row

    pattern_row = ['Coverage', coverage_mean_onestep, coverage_std_onestep, coverage_mean_twostep,coverage_std_twostep]
    all_metrics_mean_df.loc[len(all_metrics_mean_df), :] = pattern_row

    metrics_dataframes.append(all_metrics_mean_df)

    display(all_metrics_mean_df)

p = 0.5


,Metric,Onestep_MEAN,Onestep_STD,Twostep_MEAN,Twostep_STD
0,Time,0.301725,0.074693,0.460657,0.137782
1,Size,5.571429,1.057997,5.571429,1.057997
2,Ranges_Sum,4.56603,0.641354,4.574906,0.64471
3,Coverage,3.337662,4.119252,3.974026,4.948149


In [11]:
# dfi.export(all_metrics_mean_df, './saved_dataframe_image/'+dataset_name +"_"+ str(p_values[0])+'.jpg')
# all_metrics_mean_df.to_csv('./saved_dataframe_csv/'+dataset_name +"_"+ str(p_values[0])+'.csv',index=False)